In [1]:
%load_ext autoreload
%autoreload 1

import numpy as np
import theano
import theano.tensor as T
import time

import lasagne as L

import sys
sys.path.append('../')

from reddit_load import load_singles, load_pairs, get_reddit_voc

%aimport HRED

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 780 (CNMeM is enabled with initial size: 30.0% of memory, cuDNN 5105)


In [2]:
%%time
reddit_path = "/pio/data/data/reddit_sample/"

train_singles, test_singles = load_singles(path=reddit_path)
data_pairs = load_pairs(path=reddit_path)
idx_to_w, w_to_idx, voc_size, freqs = get_reddit_voc(path=reddit_path)

CPU times: user 9.62 s, sys: 109 ms, total: 9.72 s
Wall time: 9.72 s


In [3]:
%%time
hred_net = HRED.HRED(voc_size=voc_size,
                     emb_size=300,
                     lv1_rec_size=300, 
                     lv2_rec_size=300, 
                     out_emb_size=300, 
                     num_sampled=200,
                     ssoft_probs=freqs,
                     n=1)

Building the model...
Compiling theano functions...
Building a network for generating...
Done
CPU times: user 36.5 s, sys: 965 ms, total: 37.5 s
Wall time: 37.4 s


In [6]:
# hred_net.load_params('')

In [5]:
# 2ep

hred_net.train_one_epoch(data_singles, 60, log_interval=100)

Done 100 batches in 5.00s	training loss:	3.428550
Done 200 batches in 9.97s	training loss:	3.421801
Done 300 batches in 14.99s	training loss:	3.422451
Done 400 batches in 19.95s	training loss:	3.414273
Done 500 batches in 24.35s	training loss:	3.395562
Done 600 batches in 28.43s	training loss:	3.354878
Done 700 batches in 32.85s	training loss:	3.345874
Done 800 batches in 37.86s	training loss:	3.360305
Done 900 batches in 42.60s	training loss:	3.361262
Done 1000 batches in 47.48s	training loss:	3.366504
Done 1100 batches in 52.54s	training loss:	3.374706
Done 1200 batches in 57.65s	training loss:	3.379332
Done 1300 batches in 62.66s	training loss:	3.383389
Done 1400 batches in 67.66s	training loss:	3.385690
Done 1500 batches in 72.73s	training loss:	3.387937
Done 1600 batches in 77.82s	training loss:	3.391383
Done 1700 batches in 82.68s	training loss:	3.392958
Done 1800 batches in 85.33s	training loss:	3.362766
Done 1900 batches in 89.99s	training loss:	3.361663
Done 2000 batches in 94

3.3252608175728415

In [ ]:
hred_net.save_params('')